# 🚀 BERT Fine-Tuning for Text Classification
Using HuggingFace Transformers + Custom CSV Dataset (text, label)

In [1]:
!pip install transformers datasets scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.5 MB 2.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/10.5 MB 2.3 MB/s eta 0:00:05
   ------ --------------------------------- 1.6/10.5 MB 2.3 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/10.5 MB 2.1 MB/s eta 0:00:04
   --------- ------------------------------ 2.4/10.5 MB 2.1 MB/s eta 0:00:04
   ----------- ---------------------------- 2.9/10.5 MB 2.2 MB/s eta 0:00:04
   ------------- -------------------------- 3.4/10.5 MB 2.2 MB/s eta 0:00:04
   --------------- ------------------------ 3.9/10.5 MB 2.2 MB/s eta 0:00:03
   ----------------- ---------------------- 4.5/10.5 MB 2.3 MB/s eta 0:00:03
   ------------------- -------------------- 5.0/10.5 MB 2.3 MB/s eta 0:00:03
   ---------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Load your dataset (make sure you have a CSV file with columns: text,label)
dataset = load_dataset('csv', data_files='classification_data.csv')

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column('label', 'labels')
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_test = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test['train']
test_dataset = train_test['test']

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
training_args = TrainingArguments(
    output_dir='./bert-finetuned',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir='./logs',
    save_total_limit=1,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()